In [2]:
import pandas as pd
import recordlinkage as rl
from recordlinkage.index import Block
import sqlite3 as sql
import functions as fx
import core_constants as cc

In [4]:
dataset_keys = {'sports247': 'Unlinked247', 'rivals': 'UnlinkedRivals'}
for key,value in dataset_keys.items():
    SQL = """SELECT * FROM {}""".format(value)
    vars()[key] = (fx.connDBAndReturnDF(SQL)).set_index('IDYR')
    (vars()[key]).index.name = key + '_IDYR'
    (vars()[key])['IDYR'] = (vars()[key]).index.get_level_values(0)

In [6]:
indexer = rl.BlockIndex(on=['College', 'Year'])
candidate_links = indexer.index(sports247, rivals)

In [37]:
print(len(candidate_links))

7018


In [42]:
rl.write_annotation_file(
    "annotate_rivals_14.json",
    candidate_links[7001:7500],
    sports247,
    rivals,
    dataset_a_name="247 Sports",
    dataset_b_name="Rivals")

In [23]:
annotation = rl.read_annotation_file("result.json")
annotation_dict = (annotation.links).to_flat_index()

In [24]:
print(annotation_dict[0][0])

krisdike_tcu_2018


In [25]:
for record in annotation_dict:
    Values = [record[0], record[1], 2, 1, 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence)
        VALUES (?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()